In [1]:
import lazyqsar
import pandas as pd
import numpy as np
import collections
import joblib
import os

# Get all pathogens i.e. {pathogen}_{target}
PATHOGENS = sorted(os.listdir(os.path.join("..", "data")))

# Define some paths
PATH_TO_FEATURES = os.path.join("..", "output", "02_features")
PATH_TO_OUTPUT = os.path.join("..", "output", "03_baseline_models")

/home/acomajuncosa/miniconda3/envs/lazyqsar/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for pathogen in PATHOGENS:

    print(f"----------------------- PATHOGEN: {pathogen} ---------------------------")

    # Get list of tasks
    tasks = sorted(os.listdir(os.path.join("..", "data", pathogen)))

    # Get IK to MFP
    IKs = open(os.path.join(PATH_TO_FEATURES, pathogen, 'IKs_CheMeleon.txt')).read().splitlines()
    CheMeleons = np.load(os.path.join(PATH_TO_FEATURES, pathogen, "X_CheMeleon.npz"))['X']
    IK_TO_CHEMELEONS = {i: j for i, j in zip(IKs, CheMeleons)}

    break


----------------------- PATHOGEN: abaumannii_organism ---------------------------


In [3]:
# For each task
for task in tasks:

    print(f"TASK: {task}")

    # Create output_dir
    output_dir = os.path.join(PATH_TO_OUTPUT, pathogen, task.replace(".csv", ""))
    os.makedirs(output_dir, exist_ok=True)

    # Load data
    df = pd.read_csv(os.path.join("..", "data", pathogen, task))
    cols = df.columns.tolist()
    X, Y = [], []
    for ik, act in zip(df['inchikey'], df[cols[2]]):
        if ik in IK_TO_CHEMELEONS:
            X.append(IK_TO_CHEMELEONS[ik])
            Y.append(act)

    # To np.array
    X = np.array(X)
    Y = np.array(Y)

    print(X.shape)
    print(X)

    break

TASK: 1_assay_CHEMBL4296188_Inhibition_percentage_activity_percentile_10_ORGANISM_1.csv
(21482, 2048)
[[0.02986  0.01063  0.       ... 0.159    0.11743  0.005417]
 [0.       0.1103   0.       ... 0.12415  0.0616   0.      ]
 [0.003132 0.00466  0.       ... 0.2052   0.03244  0.      ]
 ...
 [0.06366  0.04065  0.04892  ... 0.02504  0.11194  0.02026 ]
 [0.01241  0.0829   0.       ... 0.2445   0.1359   0.0679  ]
 [0.2257   0.1224   0.       ... 0.1338   0.2546   0.      ]]


In [5]:
model = lazyqsar.LazyBinaryClassifier(model_type = "random_forest")
model.fit(X, Y)
preds = model.predict_proba(X)
print(preds)

Total samples: 21482, positive samples: 2144, negative samples: 19338
Maximum samples per partition: 10000, minimum samples per partition: 30
Positive proportion: 0.10
Original positive samples: 2144, total samples: 21482
Maximum samples: 10000
Sampling 2144 positive and 7856 negative samples from 10000 total samples.


  0%|          | 0/1000 [00:00<?, ?it/s]

  1%|          | 7/1000 [00:00<00:36, 26.91it/s]


All indices seen at least 1 times. Stopping sampling.
Unique sampled indices matrix shape: (8, 10000)


100%|██████████| 8/8 [01:02<00:00,  7.84s/it]


...

Best PCA score: 0.7359, No PCA score: 0.7235
PCA decision: False
Suggested zero-shot hyperparameters: {'n_estimators': 501, 'max_features': 0.24484242524861066, 'criterion': 'entropy', 'max_leaf_nodes': 1156, 'random_state': 12032022, 'verbose': 0, 'class_weight': 'balanced_subsample'}
Fitting...


[I 2025-07-22 13:29:49,104] Trial 0 finished with value: 0.7626366639308548 and parameters: {'n_estimators': 501, 'max_features': 0.24484242524861066, 'max_leaf_nodes': 1156, 'criterion': 'entropy'}. Best is trial 0 with value: 0.7626366639308548.


Best hyperparameters: {'n_estimators': 501, 'max_features': 0.24484242524861066, 'max_leaf_nodes': 1156, 'criterion': 'entropy', 'n_jobs': 8, 'random_state': 42, 'class_weight': 'balanced_subsample'}, Inner hyperparameter AUROC: 0.7626366639308548
Internal AUROC CV-0: 0.7599610146821898
Internal AUROC CV-1: 0.7702146092348847
Internal AUROC CV-2: 0.7579105807520442
Logistic regression for calibration...
Logistic regression fit done.
Average AUROC: 0.7626954015563729
Model fitted.
Fitting model on 10000 samples, positive samples: 2144, negative samples: 7856, number of features 1939
Deciding whether to use PCA for the binary classifier.


Evaluating PCA components:   0%|          | 0/3 [00:00<?, ?it/s]/home/acomajuncosa/miniconda3/envs/lazyqsar/lib/python3.11/site-packages/sklearn/preprocessing/_data.py:1085: RuntimeWarning: overflow encountered in divide
  X /= self.scale_
Evaluating PCA components:   0%|          | 0/3 [00:37<?, ?it/s]


ValueError: Input X contains infinity or a value too large for dtype('float64').